<a href="https://colab.research.google.com/github/pordeus/phd/blob/main/elm_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ELM ###

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from scipy.special import softmax

def relu(x):
    return x * (x > 0)

def sigmoid(x):
    return x / (1 + np.exp(x))
    
def linear(x):
    return x

class ELMBase:
    def __init__(self, n_hiddens=128, random_state=12, activation=linear):
        self.n_hiddens = n_hiddens
        self.rs = np.random.RandomState(random_state)
        self.activation = activation
    
    
class ELMRegressor(ELMBase):
    def __init__(self, n_hiddens=128, random_state=12):
        ELMBase.__init__(self, n_hiddens, random_state, linear)
        
    def fit(self, X, y):
        self.W = self.rs.normal(size=(X.shape[1], self.n_hiddens))
        self.b = self.rs.normal(size=(self.n_hiddens))
        y = y.reshape(-1, 1)
        
        H = self.activation(X.dot(self.W) + self.b)
        self.Beta = np.linalg.pinv(H).dot(y)
        return self
    
    def predict(self, X):
        H = self.activation(X.dot(self.W) + self.b)
        dot_product = H.dot(self.Beta)
        return dot_product
    
    
class ELMClassifier(ELMBase):
    def __init__(self, n_hiddens=128, random_state=12):
        ELMBase.__init__(self, n_hiddens, random_state, relu)
        self.output_activation = softmax
        self.encoder = OneHotEncoder()
        
    def fit(self, X, y):
        self.W = self.rs.normal(size=(X.shape[1], self.n_hiddens))
        self.b = self.rs.normal(size=(self.n_hiddens))
        y = self.encoder.fit_transform(y.reshape(-1, 1)).toarray()
        
        H = self.activation(X.dot(self.W) + self.b)
        self.Beta = np.linalg.pinv(H).dot(y)
        return self

    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)
    
    def predict_proba(self, X):
        H = self.activation(X.dot(self.W) + self.b)
        dot_product = H.dot(self.Beta)
        return self.output_activation(dot_product)

### MAIN ###

In [ ]:
import time
from sklearn.metrics import mean_squared_error
from tensorflow.keras.datasets import boston_housing
import pandas as pd

from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

In [ ]:
dataset = pd.read_excel('/content/drive/MyDrive/Doutorado/Dataset/Dataset_clinico_orto.xlsx')

In [ ]:
y = dataset['Tempo de internação'] #['Classificação de gravidade']
X = dataset.drop(['Tempo de internação'], axis=1)
Xtrain = np.array(X[:36])
Xtest = np.array(X[36:])
ytrain = np.array(y[:36])
ytest = np.array(y[36:])

In [ ]:
ytrain = ytrain.reshape(ytrain.shape[0],1)
ytest = ytest.reshape(ytest.shape[0],1)

In [ ]:
ytrain

array([[11],
       [ 7],
       [ 3],
       [10],
       [ 7],
       [37],
       [88],
       [28],
       [ 7],
       [ 2],
       [45],
       [20],
       [ 3],
       [19],
       [12],
       [15],
       [11],
       [ 8],
       [43],
       [ 8],
       [25],
       [ 3],
       [ 9],
       [13],
       [10],
       [11],
       [20],
       [ 5],
       [ 3],
       [ 5],
       [ 3],
       [13],
       [25],
       [11],
       [17],
       [ 7]])

In [ ]:
random_state = 12

In [ ]:
n_hiddens = 256

algorithms = [
    ('ELM', ELMRegressor(n_hiddens=n_hiddens, random_state=random_state)),
    ('Ridge Regression', Ridge(random_state=random_state)),
    ('Support Vector Machine', SVR()),
    ('K-Nearest Neighbors', KNeighborsRegressor()),
    ('Decision Tree', DecisionTreeRegressor(random_state=random_state)),
    ('Random Forest', RandomForestRegressor(random_state=random_state)),
    ('Perceptron (back-propagation)', MLPRegressor(hidden_layer_sizes=(n_hiddens,), random_state=random_state))
]

In [ ]:
for name, model in algorithms:
    start_time = time.perf_counter()
    model.fit(Xtrain, ytrain)
    end_time = time.perf_counter()
    
    consumed_time = round((end_time - start_time) * 1000, 2)
    print(f'{name} cost {consumed_time} miliseconds')

ELM cost 35.27 miliseconds
Ridge Regression cost 24.63 miliseconds
Support Vector Machine cost 2.27 miliseconds
K-Nearest Neighbors cost 0.38 miliseconds
Decision Tree cost 1.21 miliseconds
Random Forest cost 158.23 miliseconds


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Perceptron (back-propagation) cost 48.62 miliseconds


In [ ]:
for name, model in algorithms:
    
    train_pred = model.predict(Xtrain)
    test_pred = model.predict(Xtest)

    rmse_train = round(mean_squared_error(ytrain, train_pred, squared=False), 2)
    rmse_test = round(mean_squared_error(ytest, test_pred, squared=False), 2)

    print(f'{name} has RMSE train: {rmse_train}, RMSE test: {rmse_test}')

ELM has RMSE train: 12.22, RMSE test: 24.64
Ridge Regression has RMSE train: 12.28, RMSE test: 23.2
Support Vector Machine has RMSE train: 16.99, RMSE test: 17.95
K-Nearest Neighbors has RMSE train: 14.69, RMSE test: 17.8
Decision Tree has RMSE train: 0.0, RMSE test: 30.64
Random Forest has RMSE train: 5.87, RMSE test: 23.56
Perceptron (back-propagation) has RMSE train: 1768.3, RMSE test: 2300.2
